<a href="https://colab.research.google.com/github/yyduyuxuan/Machine-Learning-for-Data-Driven-Inventory-Replenishment-Evidence-from-the-M5-Retail-Dataset/blob/main/Base_Stock_ML_1y.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

# Val set RMSE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
val_ens = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/val_ens_adap.pkl")
val_ens.head()

Mounted at /content/drive


,item_id,store_id,dept_id,d,true,pred_store,pred_state,pred_ens,w_store,w_state,method,eta_used
0,FOODS_1_001,CA_1,FOODS_1,1370,0.0,2.285793,2.454083,2.322110,0.784202,0.215798,Ridge,0.05
1,FOODS_1_001,CA_1,FOODS_1,1371,0.0,2.861249,2.589006,2.802500,0.784202,0.215798,Ridge,0.05
2,FOODS_1_001,CA_1,FOODS_1,1372,1.0,2.100189,2.184395,2.118361,0.784202,0.215798,Ridge,0.05
3,FOODS_1_001,CA_1,FOODS_1,1373,1.0,2.374561,2.260812,2.350014,0.784202,0.215798,Ridge,0.05
4,FOODS_1_001,CA_1,FOODS_1,1374,1.0,2.036972,2.006297,2.030353,0.784202,0.215798,Ridge,0.05


In [ ]:

ml_val = val_ens[['item_id','store_id','pred_ens','true']].copy()
ml_val = ml_val.rename(columns={'pred_ens':'y_hat', 'true':'y_true'})


ml_val['unique_id'] = ml_val['item_id'].astype(str) + '_' + ml_val['store_id'].astype(str)
ml_val['model'] = 'ML_ens'
group_cols = ['unique_id','model']
ml_val.head()

,item_id,store_id,y_hat,y_true,unique_id,model
0,FOODS_1_001,CA_1,2.322110,0.0,FOODS_1_001_CA_1,ML_ens
1,FOODS_1_001,CA_1,2.802500,0.0,FOODS_1_001_CA_1,ML_ens
2,FOODS_1_001,CA_1,2.118361,1.0,FOODS_1_001_CA_1,ML_ens
3,FOODS_1_001,CA_1,2.350014,1.0,FOODS_1_001_CA_1,ML_ens
4,FOODS_1_001,CA_1,2.030353,1.0,FOODS_1_001_CA_1,ML_ens


In [ ]:
err_all = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Supervised Project/Results_Comparison/err_all.csv')
err_all.head()

,SeriesName,Demand_Type,Model,RMSE,Norm_RMSE,ME,Norm_ME,RMSSE,note
0,FOODS_1_145_CA_1,Intermittent,ML,3.287913,0.595540,-0.090838,-0.016453,1.195770,NaN
1,FOODS_1_145_CA_1,Intermittent,AutoARIMA,3.610356,0.655933,0.098940,0.017976,1.313037,NaN
2,FOODS_1_145_CA_1,Intermittent,CrostonClassic,3.783855,0.687454,0.198284,0.036024,1.376137,NaN
3,FOODS_1_145_CA_1,Intermittent,SeasonalNaive,5.056468,0.918664,0.083102,0.015098,1.838969,NaN
4,FOODS_1_145_CA_1,Intermittent,Theta,3.753326,0.681908,0.000654,0.000119,1.365034,NaN


In [ ]:
# Get 159 items

uids_159 = sorted(err_all['SeriesName'].unique())
print("uids_159:", len(uids_159))

if 'unique_id' not in ml_val.columns:
    ml_val = ml_val.copy()
    ml_val['unique_id'] = (
        ml_val['item_id'].astype(str) + '_' + ml_val['store_id'].astype(str)
    )

ml_val_159 = ml_val[ml_val['unique_id'].isin(uids_159)].copy()

print("[CHECK] selected rows:", len(ml_val_159))
print("[CHECK] cover series:", ml_val_159['unique_id'].nunique())


uids_159: 159
[CHECK] selected rows: 14310
[CHECK] cover series: 159


In [ ]:
ml_val_159.head()

,item_id,store_id,y_hat,y_true,unique_id,model
126605,FOODS_1_145,CA_1,4.393820,9.0,FOODS_1_145_CA_1,ML_ens
126606,FOODS_1_145,CA_1,5.016761,6.0,FOODS_1_145_CA_1,ML_ens
126607,FOODS_1_145,CA_1,4.417373,4.0,FOODS_1_145_CA_1,ML_ens
126608,FOODS_1_145,CA_1,4.890646,4.0,FOODS_1_145_CA_1,ML_ens
126609,FOODS_1_145,CA_1,3.655187,5.0,FOODS_1_145_CA_1,ML_ens


In [ ]:
import numpy as np
import pandas as pd

def baseline_sigma_prior_on_val(
    df: pd.DataFrame,
    group_cols: list | None = None,
    pick_model: str | None = None
) -> pd.DataFrame:

    need = {'y_hat','y_true'}
    if not need.issubset(df.columns):
        raise ValueError(f"df missing: {need - set(df.columns)}")

    if group_cols is None:
        if 'unique_id' in df.columns and 'model' in df.columns:
            group_cols = ['unique_id','model']
        else:
            raise ValueError("Need: ['unique_id','model'].")

    cols_keep = list(set(group_cols + ['y_hat','y_true']))
    df = df[cols_keep].copy()

    def _rmse(g):
        m = np.isfinite(g['y_hat']) & np.isfinite(g['y_true'])
        if m.sum() == 0:
            return pd.Series({'sigma_prior_daily': np.nan, 'n_days': 0})
        e = g.loc[m, 'y_hat'] - g.loc[m, 'y_true']
        return pd.Series({
            'sigma_prior_daily': float(np.sqrt(np.mean(e**2))),
            'n_days': int(m.sum())
        })

    out = (df.groupby(group_cols, sort=False)
             .apply(_rmse)
             .reset_index())

    return out

In [ ]:
prior_ml_159 = baseline_sigma_prior_on_val(
    ml_val_159[['unique_id','model','y_hat','y_true']],
    group_cols=['unique_id','model']
).rename(columns={'sigma_prior_daily':'sigma_prior_H'})

print(len(prior_ml_159), 'series'); display(prior_ml_159.head())

159 series


/tmp/ipython-input-1580605637.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_rmse)


,unique_id,model,sigma_prior_H,n_days
0,FOODS_1_145_CA_1,ML_ens,2.949851,90.0
1,FOODS_1_145_CA_3,ML_ens,3.620944,90.0
2,FOODS_1_145_CA_4,ML_ens,1.395500,90.0
3,FOODS_1_145_TX_1,ML_ens,2.275810,90.0
4,FOODS_1_145_TX_2,ML_ens,3.195585,90.0


# Dynamic RMSE (Weekly Update)

In [ ]:
test_ens = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/test_ens_eva_1y.pkl")
test_ens.head()

,d,date,state_id,dept_id,item_id,store_id,true,pred
0,1464,2015-01-31,CA,FOODS_1,FOODS_1_001,CA_1,3.0,3.112850
1,1465,2015-02-01,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.371410
2,1466,2015-02-02,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.454688
3,1467,2015-02-03,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.449620
4,1468,2015-02-04,CA,FOODS_1,FOODS_1_001,CA_1,0.0,2.612036


In [ ]:
if 'unique_id' not in test_ens.columns:
    test_ens = test_ens.copy()
    test_ens['unique_id'] = (
        test_ens['item_id'].astype(str) + '_' + test_ens['store_id'].astype(str)
    )

ml_test_159 = test_ens[test_ens['unique_id'].isin(uids_159)].copy()

In [ ]:
print("[CHECK] selected rows:", len(ml_test_159))
print("[CHECK] cover series:", ml_test_159['unique_id'].nunique())

[CHECK] selected rows: 57081
[CHECK] cover series: 159


In [ ]:
ml_test_159 = ml_test_159.rename(columns={'pred':'y_hat', 'true':'y_true'})
ml_test_159['model'] = 'ML_ens'
ml_test_159.head()

,d,date,state_id,dept_id,item_id,store_id,y_true,y_hat,unique_id,model
508981,1464,2015-01-31,CA,FOODS_1,FOODS_1_145,CA_1,4.0,4.880773,FOODS_1_145_CA_1,ML_ens
508982,1465,2015-02-01,CA,FOODS_1,FOODS_1_145,CA_1,3.0,4.401682,FOODS_1_145_CA_1,ML_ens
508983,1466,2015-02-02,CA,FOODS_1,FOODS_1_145,CA_1,2.0,5.056127,FOODS_1_145_CA_1,ML_ens
508984,1467,2015-02-03,CA,FOODS_1,FOODS_1_145,CA_1,3.0,4.572454,FOODS_1_145_CA_1,ML_ens
508985,1468,2015-02-04,CA,FOODS_1,FOODS_1_145,CA_1,2.0,5.310244,FOODS_1_145_CA_1,ML_ens


In [ ]:
import numpy as np
import pandas as pd

# Dynamic RMSE（baseline, test）
def dynamic_sigma_weekly(
    test_df: pd.DataFrame,
    prior_df: pd.DataFrame,
    window_days: int = 28,      # Near-term window length (days)
    update_every: str = "7D",   # weekly update
    min_obs: int = 7,
    tau: int = 14
) -> pd.DataFrame:

    need_t = {'unique_id','model','ds','y_hat','y_true'}
    need_p = {'unique_id','model','sigma_prior_daily'}
    miss_t = need_t - set(test_df.columns)
    miss_p = need_p - set(prior_df.columns)
    if miss_t:
        raise ValueError(f"test_df missing: {miss_t}")
    if miss_p:
        raise ValueError(f"prior_df missing: {miss_p}")

    # Remove duplicates
    t = test_df[['unique_id','model','ds','y_hat','y_true']].copy()
    t['ds'] = pd.to_datetime(t['ds'])
    t = t.drop_duplicates(subset=['unique_id','model','ds']).sort_values(['unique_id','model','ds'])

    p = prior_df[['unique_id','model','sigma_prior_daily']].drop_duplicates()

    keys = ['unique_id','model']
    t = t.merge(p, on=keys, how='inner')

    out = []
    for (uid, mdl), g in t.groupby(keys, sort=False):
        g = g.sort_values('ds').copy()

        # Daily squared error
        cal = pd.DataFrame({'ds': pd.date_range(g['ds'].min(), g['ds'].max(), freq='D')})
        tmp = cal.merge(g[['ds','y_hat','y_true','sigma_prior_daily']], on='ds', how='left')
        tmp['err2'] = (tmp['y_hat'] - tmp['y_true'])**2

        prior = float(tmp['sigma_prior_daily'].dropna().iloc[0])

        # Update
        updates = pd.date_range(tmp['ds'].min(), tmp['ds'].max(), freq=update_every)
        sig_points = []
        for u in updates:
            start = u - pd.Timedelta(days=window_days)
            win = tmp[(tmp['ds'] >= start) & (tmp['ds'] < u)]['err2'].dropna()
            if len(win) >= min_obs:
                rmse_recent = float(np.sqrt(win.mean()))
                w = len(win) / (len(win) + tau)
                sigma = float(np.sqrt(w * rmse_recent**2 + (1 - w) * prior**2))
            else:
                sigma = prior
            sig_points.append({'ds': u, 'sigma_day': sigma})

        sig = pd.DataFrame(sig_points)
        daily = cal.merge(sig, on='ds', how='left').sort_values('ds')

        daily['sigma_day'] = daily['sigma_day'].ffill().fillna(prior)
        daily['unique_id'] = uid
        daily['model'] = mdl
        out.append(daily[['unique_id','model','ds','sigma_day']])

    sigma_daily = pd.concat(out, ignore_index=True)


    return sigma_daily


In [ ]:
print(ml_test_159.columns.tolist())

['d', 'date', 'state_id', 'dept_id', 'item_id', 'store_id', 'y_true', 'y_hat', 'unique_id', 'model']


In [ ]:
test_df_159 = ml_test_159.copy()
test_df_159['ds'] = pd.to_datetime(test_df_159['date'])
test_df_159.head()

,d,date,state_id,dept_id,item_id,store_id,y_true,y_hat,unique_id,model,ds
508981,1464,2015-01-31,CA,FOODS_1,FOODS_1_145,CA_1,4.0,4.880773,FOODS_1_145_CA_1,ML_ens,2015-01-31
508982,1465,2015-02-01,CA,FOODS_1,FOODS_1_145,CA_1,3.0,4.401682,FOODS_1_145_CA_1,ML_ens,2015-02-01
508983,1466,2015-02-02,CA,FOODS_1,FOODS_1_145,CA_1,2.0,5.056127,FOODS_1_145_CA_1,ML_ens,2015-02-02
508984,1467,2015-02-03,CA,FOODS_1,FOODS_1_145,CA_1,3.0,4.572454,FOODS_1_145_CA_1,ML_ens,2015-02-03
508985,1468,2015-02-04,CA,FOODS_1,FOODS_1_145,CA_1,2.0,5.310244,FOODS_1_145_CA_1,ML_ens,2015-02-04


In [ ]:
test_df_159_for_sigma = test_df_159[['unique_id','model','ds','item_id','store_id','dept_id','y_hat','y_true']]

In [ ]:
prior_ml_159_req = (
    prior_ml_159
      .rename(columns={'sigma_prior_H': 'sigma_prior_daily'})
      [['unique_id','model','sigma_prior_daily']]
      .drop_duplicates())


sigma_weekly_ml = dynamic_sigma_weekly(
    test_df     = test_df_159_for_sigma,
    prior_df    = prior_ml_159_req,
    window_days = 28,
    update_every= "7D",
    min_obs     = 7,
    tau         = 14
)
display(sigma_weekly_ml.head())
print(sigma_weekly_ml.shape)

,unique_id,model,ds,sigma_day
0,FOODS_1_145_CA_1,ML_ens,2015-01-31,2.949851
1,FOODS_1_145_CA_1,ML_ens,2015-02-01,2.949851
2,FOODS_1_145_CA_1,ML_ens,2015-02-02,2.949851
3,FOODS_1_145_CA_1,ML_ens,2015-02-03,2.949851
4,FOODS_1_145_CA_1,ML_ens,2015-02-04,2.949851


(57081, 4)


In [ ]:
sigma_weekly_ml.to_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/sigma_weekly_ml_1y.csv")

#  S = µ ( L + R ) + zσ

In [ ]:
import numpy as np
import pandas as pd

Z = 1.645

def make_S_table(pred_df: pd.DataFrame,
                 sigma_df: pd.DataFrame,
                 z: float = Z,
                 yhat_col: str = 'y_hat'):
    base_keep = ['unique_id','model','ds', yhat_col]
    optional  = [c for c in ['y_true','item_id','store_id','state_id','dept_id'] if c in pred_df.columns]
    p = pred_df[base_keep + optional].copy()

    s = sigma_df[['unique_id','model','ds','sigma_day']].copy()

    out = (p.merge(s, on=['unique_id','model','ds'], how='left')
             .assign(mu_hat_H = lambda x: x[yhat_col].astype(float),
                     SS       = lambda x: z * x['sigma_day'].astype(float),
                     S        = lambda x: x[yhat_col].astype(float) + z * x['sigma_day'].astype(float))
             .sort_values(['unique_id','ds'])
             .reset_index(drop=True))

    cols = [c for c in ['unique_id','model','ds','item_id','store_id','state_id','dept_id','y_true'] if c in out.columns]
    cols += ['mu_hat_H','sigma_day','SS','S']
    return out[cols]

S_ml = make_S_table(pred_df=test_df_159_for_sigma,
                    sigma_df=sigma_weekly_ml,
                    z=1.645)

display(S_ml.head())
print(S_ml.shape)

,unique_id,model,ds,item_id,store_id,dept_id,y_true,mu_hat_H,sigma_day,SS,S
0,FOODS_1_145_CA_1,ML_ens,2015-01-31,FOODS_1_145,CA_1,FOODS_1,4.0,4.880773,2.949851,4.852505,9.733278
1,FOODS_1_145_CA_1,ML_ens,2015-02-01,FOODS_1_145,CA_1,FOODS_1,3.0,4.401682,2.949851,4.852505,9.254187
2,FOODS_1_145_CA_1,ML_ens,2015-02-02,FOODS_1_145,CA_1,FOODS_1,2.0,5.056127,2.949851,4.852505,9.908632
3,FOODS_1_145_CA_1,ML_ens,2015-02-03,FOODS_1_145,CA_1,FOODS_1,3.0,4.572454,2.949851,4.852505,9.424960
4,FOODS_1_145_CA_1,ML_ens,2015-02-04,FOODS_1_145,CA_1,FOODS_1,2.0,5.310244,2.949851,4.852505,10.162749


(57081, 11)


In [ ]:
# Round up S
S_ml['S'] = np.ceil(S_ml['S']).astype('int64')

print(S_ml['S'].describe())

n_series = S_ml['unique_id'].nunique()
print("Distinct series (unique_id):", n_series)

print(
    S_ml.groupby('unique_id')['ds'].nunique()
        .sort_values()
        .head()
)

count    57081.00000
mean        23.88476
std         45.70477
min          1.00000
25%          5.00000
50%         10.00000
75%         23.00000
max        488.00000
Name: S, dtype: float64
Distinct series (unique_id): 159
unique_id
FOODS_1_145_CA_1    359
FOODS_1_145_CA_3    359
FOODS_1_145_CA_4    359
FOODS_1_145_TX_1    359
FOODS_1_145_TX_2    359
Name: ds, dtype: int64


In [ ]:
S_ml.head()

,unique_id,model,ds,item_id,store_id,dept_id,y_true,mu_hat_H,sigma_day,SS,S
0,FOODS_1_145_CA_1,ML_ens,2015-01-31,FOODS_1_145,CA_1,FOODS_1,4.0,4.880773,2.949851,4.852505,10
1,FOODS_1_145_CA_1,ML_ens,2015-02-01,FOODS_1_145,CA_1,FOODS_1,3.0,4.401682,2.949851,4.852505,10
2,FOODS_1_145_CA_1,ML_ens,2015-02-02,FOODS_1_145,CA_1,FOODS_1,2.0,5.056127,2.949851,4.852505,10
3,FOODS_1_145_CA_1,ML_ens,2015-02-03,FOODS_1_145,CA_1,FOODS_1,3.0,4.572454,2.949851,4.852505,10
4,FOODS_1_145_CA_1,ML_ens,2015-02-04,FOODS_1_145,CA_1,FOODS_1,2.0,5.310244,2.949851,4.852505,11


In [ ]:
S_ml.to_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/ML/S_ml_1y.csv")